<h1> Install and Import

In [0]:
!pip install PyDrive
!pip install JSAnimation

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

import os
import PIL
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

from JSAnimation import IPython_display

<h1> Tunneling to TensorBoard

In [0]:
import os
import requests
import shutil
import subprocess
import tensorflow as tf

__all__ = [
  'install_ngrok', 
  'launch_tensorboard',
]

def __shell__(cmd, split=True):
  # get_ipython().system_raw(cmd)
  result = get_ipython().getoutput(cmd, split=split)
  if result and not split:
    result = result.strip('\n')
  return result

def install_ngrok(bin_dir="/tmp"):
  """ download and install ngrok on local vm instance
  Args:
    bin_dir: full path for the target directory for the `ngrok` binary
  """
  TARGET_DIR = bin_dir
  CWD = os.getcwd()
  is_grok_avail = os.path.isfile(os.path.join(TARGET_DIR,'ngrok'))
  if is_grok_avail:
    print("ngrok installed")
  else:
    import platform
    plat = platform.platform() # 'Linux-4.4.64+-x86_64-with-Ubuntu-17.10-artful'
    if 'x86_64' in plat:
      
      os.chdir('/tmp')
      print("calling wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip ..." )
      get_ipython().system_raw( "wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip" )
      print("calling unzip ngrok-stable-linux-amd64.zip ...")
      get_ipython().system_raw( "unzip ngrok-stable-linux-amd64.zip" )
      os.rename("ngrok", "{}/ngrok".format(TARGET_DIR))
      os.remove("ngrok-stable-linux-amd64.zip")
      is_grok_avail = os.path.isfile(os.path.join(TARGET_DIR,'ngrok'))
      os.chdir(TARGET_DIR)
      if is_grok_avail:
        print("ngrok installed. path={}".format(os.path.join(TARGET_DIR,'ngrok')))
      else:
        raise ValueError( "ERROR: ngrok not found, path=".format(TARGET_DIR) )
    else:
      raise NotImplementedError( "ERROR, ngrok install not configured for this platform, platform={}".format(plat))
    os.chdir(CWD)
    return
    
def launch_tensorboard(bin_dir="/tmp", log_dir="/tmp", retval=False):
  install_ngrok(bin_dir)
    
  if not tf.gfile.Exists(log_dir):  tf.gfile.MakeDirs(log_dir)
  ps = __shell__("ps -ax")
  is_tensorboard_running = len([f for f in ps if "tensorboard" in f ]) > 0
  is_ngrok_running = len([f for f in ps if "ngrok" in f ]) > 0
  print("status: tensorboard={}, ngrok={}".format(is_tensorboard_running, is_ngrok_running))

  if not is_tensorboard_running:
    get_ipython().system_raw(
        'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
        .format(log_dir)
    )
    is_tensorboard_running = True
    
  if not is_ngrok_running:
    get_ipython().system_raw('{}/ngrok http 6006 &'.format(bin_dir))
    is_ngrok_running = True
  import time
  time.sleep(3)
  retval = requests.get('http://localhost:4040/api/tunnels')
  tensorboard_url = retval.json()['tunnels'][0]['public_url'].strip()
  print("tensorboard url=", tensorboard_url)
  if retval:
    return tensorboard_url

In [0]:
# set paths
ROOT = %pwd
LOG_DIR = os.path.join(ROOT, 'log1')

# will install `ngrok`, if necessary
# will create `log_dir` if path does not exist
launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

<h2> Get Data from Drive

In [0]:
## connect to drive and Locally download the file

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1HLYgLqxFrrWy1Q58knC85jX0PGgLgYeo'})
download.GetContentFile('Assignment')

In [0]:
!unzip ./Assignment -d ./Assignment2

<h2> Dict having file names

<h3> frames[i] contains all the images in lec i

In [0]:
frames_dir = next(os.walk('./Assignment2/frames/'))[1]
frames_dir.sort()
frames = {}
for i in range(len(frames_dir)):
  a=next(os.walk('./Assignment2/frames/' + frames_dir[i] + '/'))[2]
  a.sort()
  frames[i] = ['./Assignment2/frames/' + frames_dir[i] + '/' + s for s in a]

<h1> Intial Tests

In [0]:
print(frames[2][17])
a=PIL.Image.open(frames[2][30])
print(type(a))
plt.imshow(a)

In [0]:
!apt update
!apt install tesseract-ocr
!apt install libtesseract-dev

In [0]:
from difflib import SequenceMatcher as sm

In [0]:
import cv2
import sys
import pytesseract
 
# if __name__ == '__main__':
 
#   if len(sys.argv) < 2:
#     print('Usage: python ocr_simple.py image.jpg')
#     sys.exit(1)
   
#   # Read image path from command line
#   imPath = sys.argv[1]
     
#   # Uncomment the line below to provide path to tesseract manually
#   # pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'
 
#   # Define config parameters.
#   # '-l eng'  for using the English language
#   # '--oem 1' for using LSTM OCR Engine
config = ('-l eng --oem 1 --psm 3')
 
# Read image from disk
im1 = cv2.imread(frames[2][70], cv2.IMREAD_COLOR)
im2 = cv2.imread(frames[2][69], cv2.IMREAD_COLOR)
 
# Run tesseract OCR on image
text1 = pytesseract.image_to_string(im1, config=config)
text2=pytesseract.image_to_string(im2, config=config)
 
# Print recognized text
print(frames[2][68])
print(text1)
print("LOLOLOLOLOLOLOL")
print(text2)
print(sm(None,text1,text2).ratio())

In [0]:
print(frames[2][16])